# Autoagents

Below is a complete example of how you could structure, modularize, and package a simple AutoGen-based multi-agent conversation framework.


Examples - https://microsoft.github.io/autogen/docs/Examples/#automated-multi-agent-chat

Spring AI Prompts - https://docs.spring.io/spring-ai/reference/api/prompt.html#page-title


In [ ]:
my_autogen_project/
├── autogen_agents/
│   ├── __init__.py
│   ├── base_agent.py
│   ├── assistant_agent.py
│   ├── user_proxy_agent.py
├── autogen_utils/
│   ├── __init__.py
│   ├── config_loader.py
│   ├── code_executor.py
│   ├── logger.py
├── tests/
│   ├── test_agents.py
│   ├── test_utils.py
├── scripts/
│   ├── start_agents.py
│   ├── deploy_agents.sh
├── Dockerfile
├── setup.py
├── pyproject.toml
├── README.md
└── docs/
    ├── usage.md
    ├── api_reference.md


## 2. Code Implementation
<b>autogen_agents/base_agent.py</b>

In [ ]:
from autogen_utils.logger import logger

class ConversableAgent:
    def __init__(self, name):
        self.name = name
        logger.info(f"{self.name} agent initialized.")

    def send_message(self, message):
        raise NotImplementedError("This method should be overridden by subclasses")

    def receive_message(self, message):
        raise NotImplementedError("This method should be overridden by subclasses")


<b>autogen_agents/assistant_agent.py</b>

In [ ]:
from .base_agent.py import ConversableAgent

class AssistantAgent(ConversableAgent):
    def __init__(self, name, llm_config):
        super().__init__(name)
        self.llm_config = llm_config

    def send_message(self, message):
        response = self.generate_response(message)
        return response

    def generate_response(self, message):
        # Simulate LLM response generation
        response = f"Assistant ({self.name}) generated a response to: {message}"
        return response


<b>autogen_agents/user_proxy_agent.py</b>

In [ ]:
from .base_agent.py import ConversableAgent
from autogen_utils.code_executor import DockerCommandLineCodeExecutor

class UserProxyAgent(ConversableAgent):
    def __init__(self, name, code_execution_config):
        super().__init__(name)
        self.code_executor = code_execution_config.get('executor', None)

    def send_message(self, message):
        if "execute" in message.lower():
            return self.execute_code(message)
        else:
            return f"UserProxy ({self.name}) received: {message}"

    def execute_code(self, code):
        if self.code_executor:
            result = self.code_executor.execute(code)
            return f"Execution result: {result}"
        else:
            return "No executor configured."


<b>autogen_utils/config_loader.py</b>

In [ ]:
import os

def load_llm_config():
    return {
        "model": "gpt-4",
        "api_key": os.getenv("OPENAI_API_KEY")
    }


<b>autogen_utils/code_executor.py</b>

In [ ]:
class DockerCommandLineCodeExecutor:
    def execute(self, code):
        # Simulate code execution in Docker
        return f"Executed code in Docker: {code}"


<b>autogen_utils/logger.py</b>

In [ ]:
import logging

logger = logging.getLogger("AutoGen")
logger.setLevel(logging.INFO)

handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

logger.addHandler(handler)


<b>tests/test_agents.py</b>

In [ ]:
import unittest
from autogen_agents.assistant_agent import AssistantAgent
from autogen_agents.user_proxy_agent import UserProxyAgent
from autogen_utils.code_executor import DockerCommandLineCodeExecutor

class TestAgents(unittest.TestCase):

    def setUp(self):
        # Setup common test data and configurations
        llm_config = {"config_list": [{"model": "gpt-4", "api_key": "test_api_key"}]}
        self.assistant = AssistantAgent(name="assistant", llm_config=llm_config)
        self.executor = DockerCommandLineCodeExecutor()
        self.user_proxy = UserProxyAgent(name="user_proxy", code_execution_config={"executor": self.executor})

    def test_assistant_send_message(self):
        message = "What is the current date?"
        response = self.assistant.send_message(message)
        self.assertIn("Assistant (assistant) generated a response", response)

    def test_user_proxy_send_message(self):
        message = "Please execute this Python code."
        response = self.user_proxy.send_message(message)
        self.assertIn("Executed code in Docker", response)

    def test_user_proxy_without_executor(self):
        user_proxy_no_exec = UserProxyAgent(name="user_proxy_no_exec", code_execution_config={})
        message = "Please execute this Python code."
        response = user_proxy_no_exec.send_message(message)
        self.assertEqual(response, "No executor configured.")

if __name__ == '__main__':
    unittest.main()


<b>tests/test_utils.py</b>

In [ ]:
import unittest
from autogen_utils.code_executor import DockerCommandLineCodeExecutor
from autogen_utils.config_loader import load_llm_config

class TestUtils(unittest.TestCase):

    def test_docker_command_line_code_executor(self):
        executor = DockerCommandLineCodeExecutor()
        code = "print('Hello, world!')"
        result = executor.execute(code)
        self.assertIn("Executed code in Docker", result)

    def test_load_llm_config(self):
        llm_config = load_llm_config()
        self.assertIn("model", llm_config)
        self.assertIn("api_key", llm_config)

if __name__ == '__main__':
    unittest.main()


<b>scripts/start_agents.py</b>

In [ ]:
import os
from autogen_agents.assistant_agent import AssistantAgent
from autogen_agents.user_proxy_agent import UserProxyAgent
from autogen_utils.config_loader import load_llm_config
from autogen_utils.code_executor import DockerCommandLineCodeExecutor

# Load configurations
llm_config = load_llm_config()

# Initialize agents
assistant = AssistantAgent(name="assistant", llm_config=llm_config)
code_executor = DockerCommandLineCodeExecutor()
user_proxy = UserProxyAgent(name="user_proxy", code_execution_config={"executor": code_executor})

# Start conversation
message = "What is the current date? Please execute this Python code."
response = user_proxy.send_message(message)
print(response)

response = assistant.send_message(message)
print(response)


## 3. Packaging and Setup

<b>setup.py</b>

In [ ]:
from setuptools import setup, find_packages

setup(
    name="my_autogen_project",
    version="0.1",
    packages=find_packages(),
    install_requires=[
        # List dependencies, e.g., 'requests', 'numpy'
    ],
    entry_points={
        'console_scripts': [
            'start-agents=scripts.start_agents:main',
        ],
    },
    python_requires='>=3.7',
)


<b>Dockerfile</b>

In [ ]:
FROM python:3.9-slim

WORKDIR /app

COPY . .

RUN pip install .

CMD ["start-agents"]


## 4. Running and Testing
To run the project:

<b>1. Install the package locally:</b>

In [ ]:
pip install .


<b>2. Run the agents:</b>

In [ ]:
python scripts/start_agents.py


<b>3. Build and run the Docker container:</b>

In [ ]:
docker build -t my_autogen_project .
docker run --rm my_autogen_project


<b>4. Run Tests:</b>

pytest tests/


## 5. Documentation (docs/usage.md)

# My AutoGen Project

## Usage

### Running the Agents

```bash
python scripts/start_agents.py


Running the Tests
You can run these tests using pytest or unittest from the command line:

bash
Copy code
pytest tests/
Or

bash
Copy code
python -m unittest discover tests/

*************************************************

******************************************

To work with large paragraphs of data in Python, especially when you want to manage configurations or structured data, you can use libraries such as `pydantic`, `json`, or `yaml`. Here's how you can use `pydantic` along with JSON or YAML files to handle big paragraphs of data effectively.

### Using Pydantic with JSON

1. **Install Pydantic**: If you haven't already, you can install Pydantic using pip:

   ```bash
   pip install pydantic
   ```

2. **Define a Pydantic Model**: Create a model that represents the structure of your data.

   ```python
   from pydantic import BaseModel
   from typing import List

   class ConfigData(BaseModel):
       title: str
       description: str
       paragraphs: List[str]
   ```

3. **Read Data from a JSON File**: You can read large paragraphs from a JSON file and validate it using Pydantic.

   **Example JSON file (`config.json`)**:

   ```json
   {
       "title": "Sample Title",
       "description": "This is a sample description.",
       "paragraphs": [
           "This is the first paragraph.",
           "This is the second paragraph which is a bit longer and contains more information.",
           "And here is the third paragraph."
       ]
   }
   ```

   **Reading and Validating**:

   ```python
   import json
   from pydantic import ValidationError

   # Read the JSON file
   with open('config.json', 'r') as file:
       data = json.load(file)

   # Validate and create an instance of ConfigData
   try:
       config = ConfigData(**data)
       print(config)
   except ValidationError as e:
       print("Error in configuration:", e)
   ```

### Using Pydantic with YAML

1. **Install PyYAML**: If you prefer YAML, you can use the `pyyaml` library:

   ```bash
   pip install pyyaml pydantic
   ```

2. **Define a Pydantic Model**: You can use the same model as before.

3. **Read Data from a YAML File**:

   **Example YAML file (`config.yaml`)**:

   ```yaml
   title: Sample Title
   description: This is a sample description.
   paragraphs:
     - This is the first paragraph.
     - This is the second paragraph which is a bit longer and contains more information.
     - And here is the third paragraph.
   ```

   **Reading and Validating**:

   ```python
   import yaml
   from pydantic import ValidationError

   # Read the YAML file
   with open('config.yaml', 'r') as file:
       data = yaml.safe_load(file)

   # Validate and create an instance of ConfigData
   try:
       config = ConfigData(**data)
       print(config)
   except ValidationError as e:
       print("Error in configuration:", e)
   ```

### Summary

- **Pydantic**: Use it to define the structure of your data and validate it.
- **JSON/YAML**: Use these formats to store large paragraphs or structured data externally, allowing for easy editing and maintenance.

This approach provides a clean way to manage large text data in your Python applications while ensuring that the data adheres to a defined structure.

*****************************************************************************

*****************************************************************

When implementing custom autogeneration for agents in Python, various design patterns can help you structure your code effectively and promote reusability, maintainability, and scalability. Here are some commonly used design patterns that you might consider:

### 1. **Factory Pattern**
The Factory Pattern allows you to create objects without specifying the exact class of the object that will be created. This is useful for creating different types of agents based on certain criteria.

```python
class Agent:
    def perform_action(self):
        pass

class SimpleAgent(Agent):
    def perform_action(self):
        return "Simple action performed"

class AdvancedAgent(Agent):
    def perform_action(self):
        return "Advanced action performed"

class AgentFactory:
    @staticmethod
    def create_agent(agent_type):
        if agent_type == "simple":
            return SimpleAgent()
        elif agent_type == "advanced":
            return AdvancedAgent()
        else:
            raise ValueError("Unknown agent type")

# Usage
agent = AgentFactory.create_agent("simple")
print(agent.perform_action())
```

### 2. **Strategy Pattern**
The Strategy Pattern allows you to define a family of algorithms, encapsulate each one, and make them interchangeable. This is useful for defining different strategies for agent behavior.

```python
class Strategy:
    def execute(self):
        pass

class StrategyA(Strategy):
    def execute(self):
        return "Executing Strategy A"

class StrategyB(Strategy):
    def execute(self):
        return "Executing Strategy B"

class Agent:
    def __init__(self, strategy: Strategy):
        self.strategy = strategy

    def perform_action(self):
        return self.strategy.execute()

# Usage
agent = Agent(StrategyA())
print(agent.perform_action())
```

### 3. **Observer Pattern**
The Observer Pattern allows an object (the subject) to notify other objects (observers) about changes in its state. This can be useful for agents that need to react to events or changes in the environment.

```python
class Subject:
    def __init__(self):
        self._observers = []

    def attach(self, observer):
        self._observers.append(observer)

    def notify(self):
        for observer in self._observers:
            observer.update()

class Observer:
    def update(self):
        pass

class ConcreteObserver(Observer):
    def update(self):
        print("Observer notified!")

# Usage
subject = Subject()
observer = ConcreteObserver()
subject.attach(observer)
subject.notify()
```

### 4. **Command Pattern**
The Command Pattern encapsulates a request as an object, thereby allowing for parameterization of clients with queues, requests, and operations. This is useful for implementing actions that agents can perform.

```python
class Command:
    def execute(self):
        pass

class ActionA(Command):
    def execute(self):
        return "Action A executed"

class ActionB(Command):
    def execute(self):
        return "Action B executed"

class Agent:
    def __init__(self):
        self.commands = []

    def add_command(self, command: Command):
        self.commands.append(command)

    def execute_commands(self):
        for command in self.commands:
            print(command.execute())

# Usage
agent = Agent()
agent.add_command(ActionA())
agent.add_command(ActionB())
agent.execute_commands()
```

### 5. **Decorator Pattern**
The Decorator Pattern allows behavior to be added to individual objects, either statically or dynamically, without affecting the behavior of other objects from the same class. This can be useful for enhancing agent capabilities.

```python
class Agent:
    def perform_action(self):
        return "Performing action"

class AgentDecorator:
    def __init__(self, agent: Agent):
        self.agent = agent

    def perform_action(self):
        return self.agent.perform_action()

class LoggingDecorator(AgentDecorator):
    def perform_action(self):
        result = self.agent.perform_action()
        print("Action logged.")
        return result

# Usage
agent = Agent()
decorated_agent = LoggingDecorator(agent)
print(decorated_agent.perform_action())
```

### Conclusion
These design patterns can provide a structured approach to implementing custom autogeneration for agents in Python. Depending on your specific use case, you can combine these patterns or use them individually to achieve the desired behavior and maintainability of your code.

************************************

*********************************

To enhance the Factory Pattern for different agents such as `ConversableAgent`, `AssistantAgent`, and `UserProxyAgent`, and to implement the Decorator Pattern with annotations for masking sensitive data in every conversation, we can create a comprehensive solution. Below is an implementation that covers both aspects.

### Enhanced Factory Pattern

We will define three types of agents and enhance the Factory Pattern to create instances of these agents based on input criteria.

```python
from abc import ABC, abstractmethod

# Base Agent Class
class Agent(ABC):
    @abstractmethod
    def perform_action(self):
        pass

# Conversable Agent
class ConversableAgent(Agent):
    def perform_action(self):
        return "Conversable Agent is performing an action."

# Assistant Agent
class AssistantAgent(Agent):
    def perform_action(self):
        return "Assistant Agent is assisting with a task."

# User Proxy Agent
class UserProxyAgent(Agent):
    def perform_action(self):
        return "User Proxy Agent is acting on behalf of the user."

# Agent Factory
class AgentFactory:
    @staticmethod
    def create_agent(agent_type: str) -> Agent:
        if agent_type == "conversable":
            return ConversableAgent()
        elif agent_type == "assistant":
            return AssistantAgent()
        elif agent_type == "user_proxy":
            return UserProxyAgent()
        else:
            raise ValueError("Unknown agent type")

# Usage
agent_type = "assistant"  # Example agent type
agent = AgentFactory.create_agent(agent_type)
print(agent.perform_action())
```

### Implementing the Decorator Pattern with Annotations

Next, we will implement the Decorator Pattern to mask sensitive data in conversations. We will define a base decorator and a specific decorator for masking.

```python
from functools import wraps

# Base Agent Class (same as above)
class Agent(ABC):
    @abstractmethod
    def perform_action(self):
        pass

# Concrete Agent
class SimpleAgent(Agent):
    def perform_action(self):
        return "Sensitive data: My password is 12345."

# Base Decorator Class
class AgentDecorator(Agent):
    def __init__(self, agent: Agent):
        self.agent = agent

    @abstractmethod
    def perform_action(self):
        pass

# Sensitive Data Masking Decorator
class MaskingDecorator(AgentDecorator):
    def perform_action(self):
        original_output = self.agent.perform_action()
        return self.mask_sensitive_data(original_output)

    def mask_sensitive_data(self, output: str) -> str:
        # Example of masking sensitive data
        return output.replace("12345", "*****")

# Usage
simple_agent = SimpleAgent()
masked_agent = MaskingDecorator(simple_agent)
print(masked_agent.perform_action())
```

### Explanation

1. **Factory Pattern**:
   - We created an abstract base class `Agent` and defined three specific agent types: `ConversableAgent`, `AssistantAgent`, and `UserProxyAgent`.
   - The `AgentFactory` class includes a static method `create_agent` that takes a string argument and returns the corresponding agent instance.

2. **Decorator Pattern**:
   - We defined a base decorator class `AgentDecorator` that wraps an `Agent`.
   - The `MaskingDecorator` class extends `AgentDecorator` and overrides the `perform_action` method to mask sensitive data in the output.
   - The method `mask_sensitive_data` replaces sensitive information (in this case, "12345") with asterisks.

### Conclusion

This implementation provides a structured way to manage different types of agents using the Factory Pattern while ensuring that sensitive data is masked using the Decorator Pattern. This approach enhances code maintainability and promotes the separation of concerns, making it easier to extend functionality in the future.